In [156]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import requests
import os
import pyodbc
from datetime import datetime
from sqlalchemy import create_engine, exc
from dotenv import load_dotenv

load_dotenv()

True

In [157]:
etf = 'SPY'
range = 500 #to go back to jan 23, window 10
window = 10

In [158]:
def polygon(stock_ticker, lim, wndw):
    #https://api.polygon.io/v1/indicators/sma/QQQ?timespan=day&adjusted=true&window=10&series_type=close&order=desc&limit=380&apiKey=H4Gnpv99ZxBuVG5EkE322cesxnfaPnJP
    #https://api.polygon.io/v1/indicators/sma/QQQ?timespan=day&adjusted=true&window=100&series_type=close&order=asc&limit=380&apiKey=H4Gnpv99ZxBuVG5EkE322cesxnfaPnJP
    
    # Set up the GET request
    polygon_url = 'https://api.polygon.io/v1/indicators/sma/'
    ticker = f'/{stock_ticker}'
    polygon_key = os.environ['polygon_api_key']

    params = {
            'timespan': 'day',
            'adjusted': 'true',
            'window' : '10',
            'series_type': 'close',
            'order': 'desc',
            'limit': '380',
            'apiKey': polygon_key
            }

    full_url = polygon_url + ticker 

    # Request the data
    response2 = requests.get(full_url, params)
    #polygon_data = response2.json()
    
    #polygon_data

    # Access the "results" part of the response
    #####results = polygon_data["results"]
    
    ####df_polygon = pd.DataFrame(results)
    ####df_polygon.head(5)
    
    return response2

In [159]:
def polygon(stock_ticker, lim, wndw):
    # Set up the GET request
    polygon_url = 'https://api.polygon.io/v1/indicators/sma'
    polygon_key = os.environ['polygon_api_key']
    
    # Parameters for the request
    params = {
        'timespan': 'day',
        'adjusted': 'true',
        'window': wndw,
        'series_type': 'close',
        'order': 'asc',
        'limit': lim,
        'apiKey': polygon_key
    }
    
    # Construct the full URL
    full_url = f'{polygon_url}/{stock_ticker}'
    
    # Request the data
    response = requests.get(full_url, params=params)
    polygon_data = response.json()
    results = polygon_data['results']['values']
    
    df_polygon = pd.DataFrame(results)
    #df_polygon.head(5)
    
    # Convert the Unix timestamp to datetime
    df_polygon['date'] = pd.to_datetime(df_polygon['timestamp'], unit='ms')

    # Format the datetime to the desired string format (YYYY-MM-DD)
    df_polygon['date'] = df_polygon['date'].dt.strftime('%Y-%m-%d')
    
    # Drop the unix time column
    df_polygon = df_polygon.drop(columns=['timestamp'])
    
    # Add Symbol
    df_polygon['Symbol'] = etf
    return df_polygon
    


In [160]:
# Example usage
df = polygon(etf, range, window)
df.head(5)

,value,date,Symbol
0,386.980,2022-07-22,SPY
1,388.114,2022-07-25,SPY
2,389.120,2022-07-26,SPY
3,391.341,2022-07-27,SPY
4,394.157,2022-07-28,SPY


In [161]:
# Define the connection string for Windows Authentication
server = 'MoemenLaptop'
database = 'InvestmentPortfolio'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establish the connection
try:
    conn = pyodbc.connect(connection_string)
    print("Connection successful!")
    cursor = conn.cursor()

    # Iterate over rows of the DataFrame
    for index, row in df.iterrows():
        # Example SQL insert command
        sql_command = "INSERT INTO SMA_10_Day (value, [date], symbol) VALUES (?, ?, ?)"
        
        # Execute the command
        cursor.execute(sql_command, tuple(row))

    # Commit the transaction
    conn.commit()
    
    print("Records Inserted!")

except Exception as e:
    print(f"Error: {e}")
    
    
conn.close()

Connection successful!
Records Inserted!
